In [2]:
from langgraph.graph import StateGraph, START, END 
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

import datetime, json, os


In [4]:
model = ChatOllama(model='llama3.2')

In [6]:
def calculator_tool(state):
    user_message = state["user_message"].content
    try:
        result = eval(user_message)
        reply = f"the result of {user_message} is {result}."
    except Exception as e:
        reply = f"error in calculation: {str(e)}"
    return {"messages": add_messages(state["messages"], [AIMessage(content=reply)])}

In [ ]:
def time_tool(state):
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return {"messages": add_messages(state["messages"], [AIMessage(content=f"the current date and time is: {current_time}")])}

In [ ]:
def smalltalk_node(state):
    response = model.invoke(state["messages"])
    return {"messages": add_messages(state["messages"], [AIMessage(content=response.content)])}

In [8]:
def router(state):
    last_user_message = state["user_message"].content.lower()
    if any(k in last_user_message for k in ["calculate", "what is", "solve", "+", "-", "*", "/"]):
        return "calculator"
    elif any(k in last_user_message for k in ["time", "date", "day", "now"]):
        return "time"
    else:
        return "smalltalk"

In [18]:
graph = StateGraph(dict)

graph.add_node('calculator', calculator_tool)
graph.add_node('time', time_tool)
graph.add_node('smalltalk', smalltalk_node)


graph.add_conditional_edges(START, router, {
    "calculator": "calculator",
    "time": "time",
    "smalltalk": "smalltalk"
})

graph.add_edge('calculator', END)
graph.add_edge('time', END)
graph.add_edge('smalltalk', END)




In [19]:
from langgraph.checkpoint.memory import MemorySaver


In [20]:
checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)

In [ ]:
thread_id = "chat1"   # unique id for this conversation
state = {"messages": []}

while True:
    user_input = input("you: ")
    if user_input.lower() == "exit":
        print("conversation saved. restart to continue.")
        break
    
    # add user message
    state["messages"] = add_messages(state["messages"], [HumanMessage(content=user_input)])
    state["user_message"] = HumanMessage(content=user_input)
    
    # run with checkpoint (thread_id ensures continuity)
    state = app.invoke(state, config={"configurable": {"thread_id": thread_id}})
    
    # print the ai's latest response
    print("you:", user_input)
    print("ai:", state["messages"][-1].content)

ai: How can I assist you today?
ai: I'm just a computer program, so I don't have feelings or emotions like humans do. But I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going?
ai: Hello Samir Paudel! It's nice to meet you. Is there something on your mind that you'd like to talk about, or are you looking for assistance with something specific? I'm all ears (or rather, all text).
ai: Your name is Samir Paudel! You told me that earlier when we started chatting. Is everything okay?
ai: I don't have the ability to save or store our conversation anywhere. I'm a stateless AI model, which means that each time you interact with me, it's a new session and I don't retain any information from previous conversations.

When you close this chat window or exit the platform, all of our conversation will be lost. If you want to pick up where we left off, feel free to start a new conversation with me anytime!
conversation saved. restart 